## e2eET Skeleton Based HGR Using Data-Level Fusion

In [1]:
import glob
import pickle
import numpy as np
from scipy import ndimage
from sklearn.model_selection import train_test_split

---

In [2]:
def _resize_gestures(in_gest_seqs, target_length=250):
    """Resize the time series by interpolating them to the same length"""

    out_gest_seqs = []
    for sequence in in_gest_seqs:
        zoomed_skeletons = []
        for skeleton in range(np.size(sequence, 1)):
            _zoom_skel = ndimage.zoom(sequence.T[skeleton], target_length / len(sequence), mode="reflect")
            zoomed_skeletons.append(_zoom_skel)

        out_gest_seqs.append(np.array(zoomed_skeletons).T)

    return np.array(out_gest_seqs)

In [3]:
def _write_data(data, filepath):
    """Save the dataset to a file. Note: data is a dict with keys 'X_train', ..."""

    with open(filepath, "wb") as output_file: pickle.dump(data, output_file)

In [4]:
def load_txt_gestures(type, resize_length, root):
    """
    Get the 3D pose gestures sequences, and their associated labels.
    Output:  a tuple of (gestures, str_labels, labels_14, labels_28).
    """

    # _____
    assert "DHG1428" in root, "Check that the correct dataset folder is provided!"
    assert type in ["2d", "3d"], "Data type has to be specified ['2d' / '3d']"

    # _____
    translation = {
        "gesture_1": "Grab-",
        "gesture_2": "Tap-",
        "gesture_3": "Expand-",
        "gesture_4": "Pinch-",
        "gesture_5": "Rotation_CW-",
        "gesture_6": "Rotation_CCW-",
        "gesture_7": "Swipe_Right-",
        "gesture_8": "Swipe_Left-",
        "gesture_9": "Swipe_Up-",
        "gesture_10": "Swipe_Down-",
        "gesture_11": "Swipe_X-",
        "gesture_12": "Swipe_+-",  # swapped
        "gesture_13": "Swipe_V-",  # swapped
        "gesture_14": "Shake-",
    }
    trash = [
        root,
        "skeleton_world.txt",
        "skeleton_image.txt",
        "\\",
        "_",
        "inger",
        "ubject",
        "ssai",
    ]
    pattern = (
        root + "/gesture_*/finger_*/subject_*/essai_*/skeleton_world.txt" # 3D
        if type == "3d"
        else root + "/gesture_*/finger_*/subject_*/essai_*/skeleton_image.txt" # 2D
    )
    
    # _____
    filenames = sorted(glob.glob(pattern))
    gestures = [np.genfromtxt(f) for f in filenames]
    if resize_length: gestures = _resize_gestures(gestures, target_length=resize_length)

    # _____
    labels_14 = [int(f.split("\\")[-5].split("_")[1]) for f in filenames]
    n_fingers_used = [int(f.split("\\")[-4].split("_")[1]) for f in filenames]
    labels_28 = [
        labels_14[idx] if n_fingers == 1 else 14 + labels_14[idx]
        for idx, n_fingers in enumerate(n_fingers_used)
    ]

    # _____
    str_labels = filenames.copy()
    for i in range(len(str_labels)):
        gesture_n = str_labels[i].split("\\")[-5]
        str_labels[i] = str_labels[i].replace(gesture_n, translation[gesture_n])
        for subs in trash:
            str_labels[i] = str_labels[i].replace(subs, "")

    assert len(str_labels) == len(labels_14) == len(labels_28)
    return gestures, str_labels, labels_14, labels_28

In [5]:
def load_pckl_data(filepath):
    """
    Returns hand gesture sequences (X) and their associated labels (Y).
    Each sequence has three different labels: str_labels, labels_14, and labels_28.
    """

    file = open(filepath, "rb")
    data = pickle.load(file, encoding="latin1") # change to 'latin1' to 'utf8' if the data does not load
    file.close()

    return (
        data["X_train"], data["X_valid"],
        data["train_str_labels"], data["valid_str_labels"],
        data["y_train_14"], data["y_valid_14"],
        data["y_train_28"], data["y_valid_28"],
    )

In [6]:
def create_train_valid_data(type, root, resize_length=None, seed=17711, save_path=None):
    assert type in ["2d", "3d"], "Data type has to be specified ['2d' / '3d']"
    
    # load the dataset gesture sequnences from file(s)
    gestures, str_labels, labels_14, labels_28 = load_txt_gestures(type, resize_length, root)
    print(">>> <gestures, str_labels, labels_14, labels_28> loaded successfully!")
    
    # split into train and validation subsets 
    (
        X_train, X_valid,
        train_str_labels, valid_str_labels,
        y_train_14, y_valid_14,
        y_train_28, y_valid_28,
    ) = train_test_split(gestures, str_labels, labels_14, labels_28, test_size=0.30, random_state=seed)
    print(f">>> {type} training ({X_train.shape}) and validation ({X_valid.shape}) data created.")
    
    # save the test-train data to disk
    if save_path is None: save_path = "../datasets"
    save_path = f"{save_path}/DHG1428_{type}_dictTVS_l{resize_length}_s{len(gestures)}.pckl"
        
    data = {
        "X_train": X_train, "X_valid": X_valid,
        "train_str_labels": train_str_labels, "valid_str_labels": valid_str_labels,
        "y_train_14": y_train_14, "y_valid_14": y_valid_14,
        "y_train_28": y_train_28, "y_valid_28": y_valid_28,
    }
    _write_data(data, filepath=save_path)
    print(f">>> TVS train-valid data written to <{save_path}> successfully!")

---

In [8]:
create_train_valid_data(type="3d", root="../datasets/DHG1428", resize_length=250)

>>> <gestures, str_labels, labels_14, labels_28> loaded successfully!
>>> 3d training ((1960, 250, 66)) and validation ((840, 250, 66)) data created.
>>> TVS train-valid data written to <../datasets/DHG1428_3d_dictTVS_l250_s2800.pckl> successfully!
